In [1]:
import requests
import time
import random
import yfinance as yf
from add_val import Add_Val
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [ ]:
# Data not avaliable on yfiance api
ticker_symbol = 'APPL'

# Initialize data frames
prev_close = []
open = []
bid =[]
ask = []
day_range = []
range_52 = []
volume = []
avg_vol = []
market_cap = []
beta = []
pe_ratio = []
eps = []
one_year_est = []
price = []
price_close = []

# Initialize hash table
attr = {}

# Initialize data extraction
data_value_ext = Add_Val('data-value')
text_ext = Add_Val('text')

In [ ]:
def scrape_date():
    url = 'https://finance.yahoo.com/quote/AAPL/'
    url_plot = 'https://finance.yahoo.com/chart/AAPL#eyJsYXlvdXQiOnsiaW50ZXJ2YWwiOjEsInBlcmlvZGljaXR5IjoxLCJ0aW1lVW5pdCI6Im1pbnV0ZSIsImNhbmRsZVdpZHRoIjoxLjIzMDc2OTIzMDc2OTIzMDgsImZsaXBwZWQiOmZhbHNlLCJ2b2x1bWVVbmRlcmxheSI6dHJ1ZSwiYWRqIjp0cnVlLCJjcm9zc2hhaXIiOnRydWUsImNoYXJ0VHlwZSI6Im1vdW50YWluIiwiZXh0ZW5kZWQiOmZhbHNlLCJtYXJrZXRTZXNzaW9ucyI6e30sImFnZ3JlZ2F0aW9uVHlwZSI6Im9obGMiLCJjaGFydFNjYWxlIjoibGluZWFyIiwic3R1ZGllcyI6eyLigIx2b2wgdW5kcuKAjCI6eyJ0eXBlIjoidm9sIHVuZHIiLCJpbnB1dHMiOnsiU2VyaWVzIjoic2VyaWVzIiwiaWQiOiLigIx2b2wgdW5kcuKAjCIsImRpc3BsYXkiOiLigIx2b2wgdW5kcuKAjCJ9LCJvdXRwdXRzIjp7IlVwIFZvbHVtZSI6IiMwZGJkNmVlZSIsIkRvd24gVm9sdW1lIjoiI2ZmNTU0N2VlIn0sInBhbmVsIjoiY2hhcnQiLCJwYXJhbWV0ZXJzIjp7ImNoYXJ0TmFtZSI6ImNoYXJ0IiwiZWRpdE1vZGUiOnRydWUsInBhbmVsTmFtZSI6ImNoYXJ0In0sImRpc2FibGVkIjpmYWxzZX19LCJwYW5lbHMiOnsiY2hhcnQiOnsicGVyY2VudCI6MSwiZGlzcGxheSI6IkFBUEwiLCJjaGFydE5hbWUiOiJjaGFydCIsImluZGV4IjowLCJ5QXhpcyI6eyJuYW1lIjoiY2hhcnQiLCJwb3NpdGlvbiI6bnVsbH0sInlheGlzTEhTIjpbXSwieWF4aXNSSFMiOlsiY2hhcnQiLCLigIx2b2wgdW5kcuKAjCJdfX0sInNldFNwYW4iOnsibXVsdGlwbGllciI6MSwiYmFzZSI6InRvZGF5IiwicGVyaW9kaWNpdHkiOnsiaW50ZXJ2YWwiOjEsInBlcmlvZCI6MSwidGltZVVuaXQiOiJtaW51dGUifSwic2hvd0V2ZW50c1F1b3RlIjp0cnVlLCJmb3JjZUxvYWQiOnRydWV9LCJvdXRsaWVycyI6ZmFsc2UsImFuaW1hdGlvbiI6dHJ1ZSwiaGVhZHNVcCI6eyJzdGF0aWMiOnRydWUsImR5bmFtaWMiOmZhbHNlLCJmbG9hdGluZyI6ZmFsc2V9LCJsaW5lV2lkdGgiOjIsImZ1bGxTY3JlZW4iOnRydWUsInN0cmlwZWRCYWNrZ3JvdW5kIjp0cnVlLCJjb2xvciI6IiMwMDgxZjIiLCJjcm9zc2hhaXJTdGlja3kiOmZhbHNlLCJkb250U2F2ZVJhbmdlVG9MYXlvdXQiOnRydWUsInN5bWJvbHMiOlt7InN5bWJvbCI6IkFBUEwiLCJzeW1ib2xPYmplY3QiOnsic3ltYm9sIjoiQUFQTCIsInF1b3RlVHlwZSI6IkVRVUlUWSIsImV4Y2hhbmdlVGltZVpvbmUiOiJBbWVyaWNhL05ld19Zb3JrIiwicGVyaW9kMSI6MTc1MzE5OTI4MCwicGVyaW9kMiI6MTc1MzU4MTYwMH0sInBlcmlvZGljaXR5IjoxLCJpbnRlcnZhbCI6MSwidGltZVVuaXQiOiJtaW51dGUiLCJzZXRTcGFuIjp7Im11bHRpcGxpZXIiOjEsImJhc2UiOiJ0b2RheSIsInBlcmlvZGljaXR5Ijp7ImludGVydmFsIjoxLCJwZXJpb2QiOjEsInRpbWVVbml0IjoibWludXRlIn0sInNob3dFdmVudHNRdW90ZSI6dHJ1ZSwiZm9yY2VMb2FkIjp0cnVlfX1dLCJyZW5kZXJlcnMiOltdfSwiZXZlbnRzIjp7ImRpdnMiOnRydWUsInNwbGl0cyI6dHJ1ZSwidHJhZGluZ0hvcml6b24iOiJub25lIiwic2lnRGV2RXZlbnRzIjpbXX0sInByZWZlcmVuY2VzIjp7fX0='

    # header to avoid bot detection
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
    }
    
    # Add delay to avoid rate limiting
    time.sleep(random.uniform(1, 3))

    try:
        response = requests.get(url, headers=headers, timeout=10)
        print(f"Response status: {response.status_code}")
        
        if response.status_code == 200:
            # Get data elements from html parse
            soup = BeautifulSoup(response.text, 'html.parser')
            
            prev_close_element = soup.find('fin-streamer', {'data-field': 'regularMarketPreviousClose'})
            open_element = soup.find('fin-streamer', {'data-field': 'regularMarketOpen'})
            bid_element = soup.find('span', attrs={'title': 'Bid'}).parent.find('span', class_='value yf-1b7pzha')
            ask_element = soup.find('span', attrs={'title': 'Ask'}).parent.find('span', class_='value yf-1b7pzha')
            day_range_element = soup.find('fin-streamer',{'data-field': 'regularMarketDayRange'})
            range_52_element = soup.find('fin-streamer', {'data-field': 'fiftyTwoWeekRange'})
            volume_element = soup.find('fin-streamer', {'data-field': 'regularMarketVolume'})
            avg_vol_element = soup.find('fin-streamer', {'data-field': 'averageVolume'})
            market_cap_element = soup.find('fin-streamer', {'data-field':'marketCap'})
            beta_element = soup.find('span', attrs={'title': 'Beta (5Y Monthly)'}).parent.find('span', class_='value yf-1b7pzha')
            pe_ratio_element = soup.find('span', attrs={'title': 'PE Ratio (TTM)'}).parent.find('fin-streamer')
            eps_element = soup.find('span', attrs={'title': 'EPS (TTM)'}).parent.find('fin-streamer')
            one_year_est_element = soup.find('fin-streamer', {'data-field': 'targetMeanPrice'})
            
            # get data value
            data_value_ext.get_val(prev_close_element, 'prev_close', prev_close, attr)
            data_value_ext.get_val(open_element, 'open', open, attr)
            text_ext.get_val(bid_element, 'bid', bid, attr)
            text_ext.get_val(ask_element, 'ask', ask, attr)
            data_value_ext.get_val(day_range_element, 'day_range', day_range, attr)
            data_value_ext.get_val(range_52_element, 'range_52', range_52, attr)
            data_value_ext.get_val(volume_element, 'volume', volume, attr)
            data_value_ext.get_val(avg_vol_element, 'avg_vol', avg_vol, attr)
            data_value_ext.get_val(market_cap_element, 'market_cap', market_cap, attr)
            text_ext.get_val(beta_element, 'beta', beta, attr)
            data_value_ext.get_val(pe_ratio_element, 'pe_ratio', pe_ratio, attr)
            data_value_ext.get_val(eps_element, 'eps', eps, attr)
            data_value_ext.get_val(one_year_est_element, 'one_year_est', one_year_est, attr)

        else:
            print(f"Request failed with status {response.status_code}")
            
    except requests.exceptions.RequestException as e:
        print(f"Request error: {e}")

Response status: 200
Response status 2: 200
Success! Value: 213.76
Success! Value: 214.70
Success! Value: 213.82 x 500
Success! Value: 215.97 x 300
Success! Value: 213.40 - 215.24
Success! Value: 169.21 - 260.10
Success! Value: 38,585,030
Success! Value: 53,038,719
Success! Value: 3.194T
Success! Value: 1.20
Success! Value: 33.31
Success! Value: 6.42
Success! Value: 230.62


In [4]:
# Debug print statements
print("=== DEBUGGING HASH TABLE ===")
print(f"Keys: {list(attr.keys())}")
print(f"Values: {list(attr.values())}")
print(f"Key-Value pairs: {attr}")
print(f"Specific value: {attr['prev_close']}")
print(f"Array length: {len(attr['prev_close'])}")
print("===========================")

=== DEBUGGING HASH TABLE ===
Keys: ['prev_close', 'open', 'bid', 'ask', 'day_range', 'range_52', 'volume', 'avg_vol', 'market_cap', 'beta', 'pe_ratio', 'eps', 'one_year_est']
Values: [['213.76'], ['214.70'], ['213.82 x 500'], ['215.97 x 300'], ['213.40 - 215.24'], ['169.21 - 260.10'], ['38,585,030'], ['53,038,719'], ['3.194T'], ['1.20'], ['33.31'], ['6.42'], ['230.62']]
Key-Value pairs: {'prev_close': ['213.76'], 'open': ['214.70'], 'bid': ['213.82 x 500'], 'ask': ['215.97 x 300'], 'day_range': ['213.40 - 215.24'], 'range_52': ['169.21 - 260.10'], 'volume': ['38,585,030'], 'avg_vol': ['53,038,719'], 'market_cap': ['3.194T'], 'beta': ['1.20'], 'pe_ratio': ['33.31'], 'eps': ['6.42'], 'one_year_est': ['230.62']}
Specific value: ['213.76']
Array length: 1
